# Translated Periodicity
This is when you want to transfer information from one side of the block to the other. Example would be top to bottom or left to right, etc. Each colored cube represents a block. From previous tutorials, a block consists of a set of vertices. 

![cube wireframe](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/block_wireframe.png?raw=true)




## Steps Overview
1. Find Connectivity - This splits the faces of all the blocks so that they match
2. Build the Block connectivity matrix using connectivity - Uses the split faces to determine which blocks are connected. value of 1 = connected, 0 = not searched yet, -1 = not connected
3. Search for connected faces on the z and y axis - Uses the block connectivity to find the outer connected faces. So, all the top faces for instance.
4. Find translated periodicity - Uses the connected faces e.g. the left and right, or top and bottom, or front and back. Checks the connected faces to see if any of them match faces from the other side.

# Environment Setup
This step is relatively short. Run the code below to install plot3d 

In [1]:
!pip install plot3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Lets copy some files from the GitHub over
This file is used an example of how to export the connectivity to a format where GlennHT can read. 


In [6]:
# Clone the source code for GlennOPT
!git clone https://github.com/nasa/plot3d_utilities.git

Cloning into 'plot3d_utilities'...
remote: Enumerating objects: 1898, done.
remote: Counting objects: 100% (746/746), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 1898 (delta 541), reused 580 (delta 439), pack-reused 1152
Receiving objects: 100% (1898/1898), 126.49 MiB | 26.31 MiB/s, done.
Resolving deltas: 100% (1242/1242), done.


In [7]:
# Copy the file we need
!cp plot3d_utilities/python/examples/block_test/glennht_con.py .

## GlennHT Connectivity Format
The glennht connectivity file is organized in 2 sections. The first section contains all the matching faces. The first number is the number of face pairs.

![Glennht connectivity format](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/glennht_connectivity.png?raw=true)

The second section show below shows the nonconnected faces also known as `outer_faces` in the code. 

![Glennht connectivity format](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/glennht_connectivity2.png?raw=true)

## Import some headers
The code below are the functions from plot3d whcih will be used to find connectivity and periodicity. 

In [8]:
import sys, os, pickle
import numpy as np
from plot3d import read_plot3D, connectivity_fast,translational_periodicity, block_connection_matrix, find_bounding_faces
from plot3d import outer_face_dict_to_list, match_faces_dict_to_list

In [9]:
#%% Find connectivity 
def dump_data(data):
    with open('block_data.pickle','wb') as f:
        pickle.dump(data,f)

def read_data():
    with open('block_data.pickle','rb') as f:
        return pickle.load(f)

## Download the Geometry
Run the code below to download the geometry into your colab environment

In [10]:
!wget https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/iso65_64blocks.xyz

--2023-02-14 15:16:44--  https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/iso65_64blocks.xyz
Resolving nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)... 52.216.219.249, 52.216.107.132, 54.231.134.33, ...
Connecting to nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)|52.216.219.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3773956 (3.6M) [binary/octet-stream]
Saving to: ‘iso65_64blocks.xyz.1’

iso65_64blocks.xyz. 100%[===================>]   3.60M  --.-KB/s    in 0.1s    

2023-02-14 15:16:44 (24.8 MB/s) - ‘iso65_64blocks.xyz.1’ saved [3773956/3773956]



# Importing the mesh 

In [11]:
blocks = read_plot3D('iso65_64blocks.xyz',True)

Checking connections block 32 with 41: 100%|██████████| 16/16 [00:00<00:00, 19.52it/s]



Periodic x


Checking connections block 27 with 63: 100%|██████████| 16/16 [00:00<00:00, 24.89it/s]



Periodic y


Checking connections block 45 with 63: 100%|██████████| 16/16 [00:00<00:00, 18.84it/s]



Periodic z
Left faces missing
Right faces missing


[]

## Find the Connectivity

In [21]:
print('Finding connectivity')
face_matches, outer_faces = connectivity_fast(blocks)
[m.pop('match',None) for m in face_matches] # Remove the dataframe
print('Organizing split and outerfaces')
all_faces = match_faces_dict_to_list(blocks,face_matches)
all_faces.extend(outer_face_dict_to_list(blocks,outer_faces))
all_faces = [m.to_dict() for m in all_faces]
data = {
            "face_matches":face_matches, 
            "outer_faces":outer_faces,
            "all_faces":all_faces
        }
dump_data(data)

Finding connectivity
gcd to use 16


Checking connections block 62 with 63: 100%|██████████| 363/363 [00:27<00:00, 13.19it/s]


Organizing split and outerfaces


## Create the Block to Block connection matrix
It's important to find the connectivity first because it splits up all the partial matching faces. Those partial matching faces are then used to build a block to block connection matrix. This matrix is essential because it ultimately speeds up finding connected faces - useful for defining boundary conditions.

In [22]:
print('Creating block connection matrix')
c = block_connection_matrix(blocks,all_faces)
data["connectivity_matrix"]=c
dump_data(data)

Creating block connection matrix


Building block to block connectivity matrix: checking 62: 100%|██████████| 2016/2016 [00:06<00:00, 330.71it/s]


## Find bounding faces
This finds for example all the faces on the right side of the block. Bounded axes are passed as "x","y",or "z".

![bounded faces](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/block_bounded_faces.png?raw=true)

In [23]:
data = read_data()    
all_faces = data['all_faces']
connectivity_matrix = data['connectivity_matrix']

#%% Find bounding Faces
forward_bound, backward_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"x")
lower_bound, upper_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"z")
left_bound, right_bound,_,_ = find_bounding_faces(blocks,connectivity_matrix,all_faces,"y")
data['lower_bound'] = lower_bound
data['upper_bound'] = upper_bound
data['left_bound'] = left_bound
data['right_bound'] = right_bound
dump_data(data)

## Find Periodicity
Now that we have the bounded faces, we can search for peridocity. In this example, we find periodicity in all directions: x, y, and z.

In [24]:
#%% Use bounding faces to find periodicity
data = read_data()
lower_bound = data['lower_bound']; upper_bound = data['upper_bound']
left_bound = data['left_bound']; right_bound = data['right_bound']
x_periodic_faces_export, periodic_faces = translational_periodicity(blocks,forward_bound,backward_bound,translational_direction='x')
y_periodic_faces_export, periodic_faces = translational_periodicity(blocks,left_bound,right_bound,translational_direction='y')
z_periodic_faces_export, periodic_faces = translational_periodicity(blocks,lower_bound,upper_bound,translational_direction='z')
data['x_periodic'] = x_periodic_faces_export
data['z_periodic'] = z_periodic_faces_export
data['y_periodic'] = y_periodic_faces_export
dump_data(data)


Checking connections block 32 with 41: 100%|██████████| 16/16 [00:00<00:00, 19.21it/s]



Periodic x


Checking connections block 27 with 63: 100%|██████████| 16/16 [00:00<00:00, 18.16it/s]



Periodic y


Checking connections block 45 with 63: 100%|██████████| 16/16 [00:01<00:00, 14.04it/s]



Periodic z


## Exporting to GlennHT Format
Because, why not.

In [16]:
data = read_data()
matched_faces = data['x_periodic']
matched_faces.extend(data['y_periodic'])
matched_faces.extend(data['z_periodic'])
matched_faces.extend(data['face_matches'])

# in this case we only have matched faces and no outer faces but lets filter it anyways
outer_faces = data['outer_faces']
outer_faces = outer_face_dict_to_list(blocks,outer_faces)
matched_faces = list(set(match_faces_dict_to_list(blocks,matched_faces)))
outer_faces = [o for o in outer_faces if o not in matched_faces]

print(f"Number of outer_faces: {len(outer_faces)}") # this should be 0

Number of outer_faces: 122


In [20]:
outer_faces

[blk: 1 [16,0,0,16,16,16],
 blk: 4 [16,0,0,16,16,16],
 blk: 5 [16,0,0,16,16,16],
 blk: 7 [16,0,0,16,16,16],
 blk: 8 [16,0,0,16,16,16],
 blk: 9 [16,0,0,16,16,16],
 blk: 10 [16,0,0,16,16,16],
 blk: 12 [16,0,0,16,16,16],
 blk: 13 [16,0,0,16,16,16],
 blk: 14 [16,0,0,16,16,16],
 blk: 39 [0,0,0,0,16,16],
 blk: 40 [0,0,0,0,16,16],
 blk: 41 [0,0,0,0,16,16],
 blk: 32 [0,0,0,0,16,16],
 blk: 33 [0,0,0,0,16,16],
 blk: 35 [0,0,0,0,16,16],
 blk: 36 [0,0,0,0,16,16],
 blk: 37 [0,0,0,0,16,16],
 blk: 38 [0,0,0,0,16,16],
 blk: 31 [0,0,16,16,16,16],
 blk: 32 [0,0,16,16,16,16],
 blk: 34 [0,0,16,16,16,16],
 blk: 37 [0,0,16,16,16,16],
 blk: 39 [0,0,16,16,16,16],
 blk: 41 [0,0,16,16,16,16],
 blk: 43 [0,0,16,16,16,16],
 blk: 45 [0,0,16,16,16,16],
 blk: 47 [0,0,16,16,16,16],
 blk: 57 [0,0,16,16,16,16],
 blk: 36 [0,0,0,16,16,0],
 blk: 38 [0,0,0,16,16,0],
 blk: 44 [0,0,0,16,16,0],
 blk: 46 [0,0,0,16,16,0],
 blk: 48 [0,0,0,16,16,0],
 blk: 50 [0,0,0,16,16,0],
 blk: 52 [0,0,0,16,16,0],
 blk: 54 [0,0,0,16,16,0],
 blk

In [19]:
data['outer_faces']

[{'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 1,
  'block_index': 1},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 2,
  'block_index': 4},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 3,
  'block_index': 5},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 4,
  'block_index': 7},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 5,
  'block_index': 8},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 6,
  'block_index': 9},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 7,
  'block_index': 10},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'id': 8,
  'block_index': 12},
 {'IMIN': 16,
  'JMIN': 0,
  'KMIN': 0,
  'IMAX': 16,
  'JMAX': 16,
  'KMAX': 16,
  'i